# Generating music with Markov Chains using real music as training data

## Import All Required Libraries

- **`pretty_midi`**: A library for handling MIDI files and musical notes in Python. It is used for loading MIDI files, converting note names to numbers, and saving MIDI files.
- **`random`**: Used for generating random choices, such as selecting the next note in the sequence.
- **`os`**: Provides functions to interact with the operating system, such as creating directories.
- **`pygame`**: A library for creating games. It is used to play the MIDI file sounds.
- **`time`**: Used to pause execution while the music is playing.

In [ ]:
import pretty_midi as pm
from collections import defaultdict
import random
import os
import pygame
import time

## Extract Notes from a MIDI File

The function *`extract_notes_from_midi()`* extracts notes from a MIDI file:
1. The *`extracted_notes`* list will store all extracted notes from the imported MIDI file;
2. The MIDI file is loaded and stored int *`midi_data`*;
3. All the instrument data is extracted by iterating through the MIDI file data. The extracted notes are appended to the *`extracted_notes`* list;
4. The function returns the *`extracted_notes`* list, containing all the extracted data from the MIDI file.

After providing the MIDI file path the function is called and the extracted notes are saved as a variable (*`notes`*).

In [ ]:
def extract_notes_from_midi(midi_file_path):
    extracted_notes = []
    
    # Load the MIDI file
    midi_data = pm.PrettyMIDI(midi_file_path)
    
    # Extract notes from all instruments
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            extracted_notes.append(pm.note_number_to_name(note.pitch))
    
    return extracted_notes

In [ ]:
# MIDI file path
midi_path = "midi_files/Never-Gonna-Give-You-Up-2.mid"

# Calling the extract_notes_from_midi() function and storing all the extracted notes
notes = extract_notes_from_midi(midi_path)

## Build Transformation Matrix

The function *`build_transition_matrix()`* creates a transition matrix from a sequence of music notes:
1. *`transition_matrix_new`*: Dictionary to store lists of possible next note, to the current note (key);
2. Iterating through the list of music notes. Each subsequent note is appended to the list of the current note in the matrix.
3. After iterating through all the notes the *`transition_matrix_new`* dictionary is returned.

After calling the function the *`transition_matrix_new`* dictionary is stored as a variable (*`transition_matrix`*)

In [ ]:
def build_transition_matrix(music_notes):
    transition_matrix_new = defaultdict(list)
    
    for i in range(len(music_notes) - 1):
        current_note = music_notes[i]
        next_note = music_notes[i + 1]
        
        transition_matrix_new[current_note].append(next_note)
    
    return transition_matrix_new

In [ ]:
transition_matrix = build_transition_matrix(notes)

## Generate Music

The function *`generate_music()`* creates a sequence of music notes using the transition matrix:
1. *`music`*: List containing all generated notes. The first note inside is the starting note (manually provided);
2. *`current_note`*: Variable, determining the current note;
3. Iterating through the list of notes:
    1. The note following the *`current_note`* is determined randomly (*`next_note`*);
    2. The *`next_note`* is appended to the *`music`* list;
    3. The *`current_note`* takes the value of the *`next_note`*.
4. After iterating through all the notes the *`music`* list is returned.

Before calling the function, the starting note is determined randomly from all the notes.
After calling the function, and providing the required parameters (the transition matrix, the starting note and the number of notes to e generated, the default=20) the generated music is stored as a variable (*`generated_music`*). 

In [ ]:
def generate_music(matrix, start_note, length=50):
    music = [start_note]
    current_note = start_note
    
    for _ in range(length - 1):
        next_note = random.choice(matrix[current_note])
        music.append(next_note)
        current_note = next_note
    
    return music

In [ ]:
starting_note = random.choice(notes)
generated_music = generate_music(transition_matrix, starting_note, length=20)

## Save Generated Music

The function *`save_generated_music_as_midi()`* saves the generated music sequence as a MIDI file:
1. *`midi_data`*: Create a new PrettyMIDI object;
2. *`instrument`*: Create a new PrettyMIDI instrument and select its type;
3. *`start_time`* & *`duration`*: Set start time of the note & set its duration;
4. Iterate through the notes of the generated music and append it to the *`instrument`*'s notes;
5. *`instrument`*'s notes are appended to the *`midi_data`* variable;
6. The *`midi_data`* is saved at the provided file save path.

In [ ]:
def save_generated_music_as_midi(gen_music, midi_save_path):
    midi_data = pm.PrettyMIDI()
    instrument = pm.Instrument(program=0)  # program=0 is Acoustic Grand Piano
    
    start_time = 0.0
    duration = 0.5
    
    for note_name in gen_music:
        note_number = pm.note_name_to_number(note_name)
        note = pm.Note(velocity=50,   # Volume Level
                                pitch=note_number, 
                                start=start_time, 
                                end=start_time + duration)
        instrument.notes.append(note)
        
        start_time += duration
    
    midi_data.instruments.append(instrument)
    midi_data.write(midi_save_path)

In [ ]:
# Set folder to save MIDI file in
folder_path = "midi_files_generated"

# Check if the provided folder exists, and if it doesn't - create it
os.makedirs(folder_path, exist_ok=True)

# Set MIDI file name
midi_filename = "markov_music_inspired.mid"

# Set output MIDI file path
output_midi_file_path = os.path.join(folder_path, midi_filename)

# Save the generated music as MIDI file
save_music = save_generated_music_as_midi(generated_music, output_midi_file_path)

## Play Generated Music

To play the MIDI file is required to: 
1. Initialize the Pygame library (*`pygame.init()`*);
2. Initialize the Pygame mixer module for playing sound (*`pygame.mixer.init()`*);
3.  Load the generated MIDI file (*`pygame.mixer.music.load(midi_filename)`*);
4. Start playing the MIDI file (*`pygame.mixer.music.play()`*);
5. Wait until the music finishes playing (*`while pygame.mixer.music.get_busy()`*);
6. Quit Pygame (*`pygame.quit()`*).

In [ ]:
def play_midi_file(midi_file):
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(midi_file)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        time.sleep(1)

    pygame.quit()


In [ ]:
# Play the generated MIDI file
play_midi_file(output_midi_file_path)

## Credits

1. MIDI Files Downloaded from [BitMidi](https://bitmidi.com)